In [1]:
from Series import Speed, Direction
from Time import Hours
from DataFrame import PriceDemand, Cities
from config import COLUMNS

import pandas as pd
from datetime import date, datetime, time, timedelta
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

In [2]:
cities_df = Cities()
price_df = PriceDemand()
joined_df = cities_df.join_price(price_df.get_grouped())

In [3]:
from Time import Hours
Hours(price_df[price_df["PRICESURGE"]]["SETTLEMENTDATE"].apply(lambda stamp: f"{stamp.hour}:{stamp.minute}:{stamp.second}")).seconds_from_midnight()

8927          0
139497     3000
139498     3300
55514     36900
71933     37800
          ...  
60924     18300
89471     28800
98111     28800
59521     29400
92946     34500
Name: SETTLEMENTDATE, Length: 205, dtype: int64

price surge for one day
price surge for 9am and 3pm seperately 

price_surge_score_3pm = for all price surges in that day, take sum(1 / | time of surge - 3pm | )


fake_value_lookup = {"Evaporation" : [row_index1, row_index2, ... ]}

testing correlations between features


test correlation of features with a label 


1) process features, normalise, fill missing etc..

2) test different approaches for choosing a label



